In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Jupyter BigQuery Storage')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [ ]:
table = "res-nbcupea-dev-ds-sandbox-001.silverTables.SILVER_VIDEO"
df = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()

In [4]:
%%bquery

SELECT * FROM silverTables.SILVER_VIDEO LIMIT 100

UsageError: Cell magic `%%bquery` not found.


In [ ]:
DECLARE holdoutcampaign_1  string;
DECLARE holdoutcampaign_2  string;
DECLARE holdoutcampaign_3  string;

DECLARE tenure_threshold_date date; -- profiles must be signed up before this day for 7 days of viewing data.

SET tenure_threshold_date = '2020-09-16'; -- 6/1 last day for signup. guarantees at least 7 days of viewing data for all users in analysis.

SET holdoutcampaign_1 = 'holdoutgroup_control';
SET holdoutcampaign_2 = 'holdoutgroup_paidsoc';
SET holdoutcampaign_3 = 'holdoutgroup_paiddis';

/**************************************************************************************************
Grab the mp tracking ids from all of the campaign holdout mparticle files.
The mp tracking id is the identity_value when the identity_type is 'other_5'.
**************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist` AS
SELECT identity_value,
      holdoutcampaign_1 AS audiencecohort
      FROM  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.all_media_holdout`
      WHERE
      identity_type = 'other_5'
UNION ALL
SELECT identity_value,
      holdoutcampaign_2 AS audiencecohort
      FROM  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.paid_social_holdout`
      WHERE
      identity_type = 'other_5'
UNION ALL
SELECT identity_value,
      holdoutcampaign_3 AS audiencecohort
      FROM  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.paid_display_holdout`
      WHERE
      identity_type = 'other_5';

/**************************************************************************************************
Append SDP user id information to each mp tracking id in the holdout masterlist.
Add profileid, mpid, adobetrackingid, and their cohortname for each user in holdout group list.
**************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo` AS
SELECT  a.ProfileID,
        a.ExternalProfilerID as mParticleID,
        b.AudienceCohort,
        c.AdobeTrackingID as aid,
    FROM `nbcu-sdp-prod-003.sdp_persistent_views.CustomerKeysMapping` a
    INNER JOIN --
      (SELECT  map.ProfileID,
               map.ExternalProfilerID as mparticleid,
               list.AudienceCohort
        FROM `nbcu-sdp-prod-003.sdp_persistent_views.CustomerKeysMapping` map
        INNER JOIN  -- only grab users in this audience holdout list, note: identity_value is the mptrackingid
         `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist` list
        ON map.ExternalProfilerID = list.identity_value) b
    ON a.ExternalProfilerID = b.mparticleid
    LEFT JOIN --  attach the adobetrackingid to identify audience members in adobe viewing table
        (SELECT   profileid AS profileid_b,
                  ExternalProfilerID AS AdobeTrackingID
            FROM
                   `nbcu-sdp-prod-003.sdp_persistent_views.CustomerKeysMapping`
            WHERE
                      PartnerOrSystemId= 'trackingid') c
            ON a.ProfileID = c.profileid_b;

/**************************************************************************************************
Append each profile's signup date to the table to account for profile tenure in analysis (late v early adopters)
**************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo_signup_dates` AS
SELECT *
FROM (
  SELECT a.*,
  b.signup_date
  FROM (
    SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo`
  ) a
LEFT JOIN
(SELECT * FROM (
        (SELECT DISTINCT HouseholdID,
                  cast(min(datetime(targetedoptindate, 'America/New_York')) as date) as signup_date
                  FROM `nbcu-sdp-prod-003.sdp_persistent_views.AccountView` group by householdid
                  ) ) )  b
ON a.ProfileID = b.HouseholdID
WHERE
signup_date < tenure_threshold_date -- must meet tenure requirement
);


In [ ]:
-- create an account tenure threshold in order to analyze a full 7 days of viewership for all viewers
DECLARE tenure_threshold_date date;
DECLARE campaign_start_date DATE DEFAULT '2020-07-29'; -- 6/1 last day

-- user signup needs to be < 6/2 for 7 days of viewing data. 6/1 max signupdate
SET tenure_threshold_date = '2020-09-16'; -- 6/1 last day


/********************************************************************************************************
  Make the targetable cohort for All Paid Media
*******************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_control` AS 
SELECT HouseholdID,
       'targetablegroup_control' AS audiencecohort
              FROM (  
                  SELECT DISTINCT HouseholdId
                  FROM `nbcu-sdp-prod-003.sdp_persistent_views.AccountView`
                  WHERE 
                  CAST(datetime(targetedoptindate, 'America/New_York') as date) < tenure_threshold_date
                ) all_householdids 
FULL OUTER JOIN 
         (SELECT DISTINCT PROFILEID 
                  FROM 
                  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo_signup_dates`
                  WHERE
                  audiencecohort = 'holdoutgroup_control'-- not in all paid media holdout
                  OR
                  audiencecohort = 'holdoutgroup_paidsoc' -- not in paid social holdout
                  OR
                  audiencecohort = 'holdoutgroup_paiddis'  -- not in paid display holdout     
                  OR
                  audiencecohort = 'holdoutgroup_brazeco'  -- not in email holdout
                  )  holdouts
  ON all_householdids.householdid  = holdouts.profileid
  WHERE
  holdouts.profileid IS NULL; -- keeps ids not matched to a holdout id.
  

/********************************************************************************************************
  Make the targetable cohort for Paid Social
*******************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_paidsoc` AS
SELECT HouseholdID,
       'targetablegroup_paidsoc' as audiencecohort
        FROM (  
        SELECT DISTINCT HouseholdId 
            FROM `nbcu-sdp-prod-003.sdp_persistent_views.AccountView`
            WHERE
            CAST(datetime(targetedoptindate, 'America/New_York') as date) < tenure_threshold_date
        ) all_householdids 
FULL OUTER JOIN (SELECT  
                  DISTINCT PROFILEID
                  FROM 
                  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo_signup_dates`
                  WHERE
                  audiencecohort = 'holdoutgroup_control'-- not in all paid media holdout
                  OR
                  audiencecohort = 'holdoutgroup_paidsoc')  holdouts --  not in paid social holdout group
  ON all_householdids.householdid  = holdouts.profileid
  WHERE
  holdouts.profileid IS NULL; -- keeps ids not matched to a holdout id

/********************************************************************************************************
  Make the targetable cohort for Paid Display
*******************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_paiddis` AS
SELECT HouseholdID,
       'targetablegroup_paiddis' as audiencecohort 
       FROM (  
            SELECT DISTINCT HouseholdId 
            FROM `nbcu-sdp-prod-003.sdp_persistent_views.AccountView`
            WHERE 
            CAST(datetime(targetedoptindate, 'America/New_York') as date) < tenure_threshold_date          
) all_householdids 
FULL OUTER JOIN ( SELECT  
                  DISTINCT PROFILEID
                  FROM 
                  `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo_signup_dates`
                  WHERE
                  audiencecohort = 'holdoutgroup_control'-- not in all paid media holdout
                  OR
                  audiencecohort = 'holdoutgroup_paiddis')  holdouts -- not in paid display holdout group
  ON all_householdids.householdid  = holdouts.profileid
  WHERE
  holdouts.profileid IS NULL; -- keeps ids not matched to holdout records
 
/********************************************************************************************************
 Union all campaign's targetable audiences lists
********************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_masterlist` AS
select Householdid AS profileid,
                  AudienceCohort 
FROM (
SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_control`
UNION ALL
SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_paidsoc`
UNION ALL
SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_paiddis`
);

/********************************************************************************************************
 Append user ids with their adobe tracking ids ("aid"'s) for table with all targetable user info 
 - note:  all but ~1100 of the targetable profile ids have adobetrackingids. make sure to only grab nonnull.
********************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_masterlist_userinfo` AS
SELECT  b.ProfileID, 
        b.ExternalProfilerID AS aid, 
        a.AudienceCohort,
        FROM  
        (
       SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_masterlist`
       )  a
   LEFT JOIN
        ( SELECT   profileid AS profileid, 
                  ExternalProfilerID
            FROM 
                   `nbcu-sdp-prod-003.sdp_persistent_views.CustomerKeysMapping`
            WHERE 
                      PartnerOrSystemId= 'trackingid'
           ) b
ON 
a.ProfileID = b.profileid
WHERE
b.profileid is not null; -- (1066 people are without adobetrackingsids who got peacock access early before launch.)


/*************************************************************************************************************
Union the targetable and holdout master tables for an  ALL audience cohort table: 
    >> columns in targetable master table: aid, profileid, audiencecohort 
    >> columns in holdout master table be: aid, profileid, audiencecohort
**************************************************************************************************************/
CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.all_audiencecohort_table` AS 
SELECT aid, profileid, audiencecohort FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.holdoutgroup_masterlist_userinfo_signup_dates`
WHERE audiencecohort <> 'holdoutgroup_brazeco' -- braze holdout was used to make a targetable. braze not to be included in analysis.
UNION ALL
SELECT aid, profileid, audiencecohort FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.targetablegroup_masterlist_userinfo`;

/*************************************************************************************************************
  Append all users in the audience cohorts with their account sign up dates for early vs late adopters distinction.
***************************************************************************************************************/

CREATE OR REPLACE TABLE `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.all_audiencecohort_table_signup_dates` AS
SELECT *,
  CASE
    WHEN signup_date < campaign_start_date THEN "early"
    WHEN signup_date >= campaign_start_date THEN "late"
--       WHEN signup_date >= CAST('2020-04-15' AS DATE) AND signup_date <=  CAST('2020-04-21' AS DATE) THEN "early_415" 
--       WHEN signup_date >= CAST('2020-04-22' AS DATE) AND signup_date <=  CAST('2020-04-28' AS DATE) THEN "early_422" 
--       WHEN signup_date >= CAST('2020-04-29' AS DATE) AND signup_date <=  CAST('2020-05-05' AS DATE) THEN "late_429" 
--       WHEN signup_date >= CAST('2020-05-06' AS DATE) AND signup_date <=  CAST('2020-05-12' AS DATE) THEN "late_56" 
--       WHEN signup_date >= CAST('2020-05-13' AS DATE) AND signup_date <=  CAST('2020-05-19' AS DATE) THEN "late_513" 
--       WHEN signup_date >= CAST('2020-05-20' AS DATE) AND signup_date <=  CAST('2020-05-26' AS DATE) THEN "late_520" 
--       WHEN signup_date >= CAST('2020-05-27' AS DATE) AND signup_date <=  CAST('2020-06-02' AS DATE) THEN "late_527" 
   ELSE "not_in_cohort"  
  END AS adopter_group -- create an adopter_group to toggle for early vs late signups depending on account tenure."late"=signup after campaign start date
FROM (
  SELECT a.*, 
  b.signup_date
  FROM (
    SELECT * FROM `res-nbcupea-dev-ds-sandbox-001.dg_sandbox.all_audiencecohort_table`
  ) a
LEFT JOIN
(SELECT * FROM (
        (SELECT DISTINCT HouseholdID,
                  cast(min(datetime(targetedoptindate, 'America/New_York')) as date) as signup_date 
                  FROM `nbcu-sdp-prod-003.sdp_persistent_views.AccountView` group by householdid
                  ) )  ) b          
ON a.ProfileID = b.HouseholdID );
